In [ ]:
!pip install -q gradio git+https://github.com/openai/whisper.git

In [ ]:
import os
import requests
import gradio as gr
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
import whisper
print('done')

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
model = whisper.load_model("small")

In [ ]:
client = OpenAI()  

def create_prompt(transcript):
    max_length = 26000
    if len(transcript) > max_length:
        transcript = transcript[:max_length] + "..."

    return [
        {
            "role": "system",
            "content": "Jesteś asystentem, który tworzy strukturalne notatki na podstawie transkrypcji. "
                       "Wygeneruj notatki w następującym formacie:\n\n"
                       "# Podsumowanie\n[krótkie podsumowanie]\n\n"
                       "# Kluczowe punkty\n- [punkt 1]\n- [punkt 2]\n\n"
                       "# Wnioski\n[główne wnioski]\n\n"
                       "# Działania do podjęcia\n- [akcja 1]\n- [akcja 2]\n\n"
                       "Odpowiadaj TYLKO w języku polskim i używaj formatu markdown."
        },
        {
            "role": "user",
            "content": f"Stwórz notatki z tej transkrypcji:\n\n{transcript}"
        }
    ]

def generate_notes_safe(transcript):
    try:
        print("🔄 Rozpoczynam generowanie notatek...")

        messages = create_prompt(transcript)

        if len(transcript) > 26000:
            return "❌ Transkrypcja jest za długa. Skróć nagranie."

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )

        print("✅ Notatki wygenerowane pomyślnie!")
        summary = response.choices[0].message.content.strip()
        return markdown.markdown(summary)
        
    except Exception as e:
        error_msg = f"❌ Błąd podczas generowania notatek: {str(e)}"
        print(error_msg)
        return error_msg

def transcribe_and_analyze(audio_path, generate_notes):
    if audio_path is None or not os.path.exists(audio_path):
        return "❌ Nie wybrano pliku audio.", ""

    try:
        print("🎙️ Rozpoczynam transkrypcję...")

        result = model.transcribe(audio_path, language="pl")
        transcript = result.get("text", "")

        if not transcript:
            return "❌ Nie udało się przeprowadzić transkrypcji.", ""

        print(f"✅ Transkrypcja ukończona: {len(transcript)} znaków")

        notes = ""
        if generate_notes:
            if len(transcript) < 50:
                notes = "❌ Transkrypcja jest za krótka do utworzenia notatek."
            else:
                notes = generate_notes_safe(transcript)

        return transcript or "", notes or ""

    except Exception as e:
        error_msg = f"❌ Błąd: {str(e)}"
        print(error_msg)
        return error_msg, ""

with gr.Blocks() as ui:
    with gr.Row():
        audio_input = gr.Audio(type="filepath", label="Wgraj lub nagraj audio", format="wav")
    
    with gr.Row():
        create_notes = gr.Checkbox(label="Utwórz notatki ze spotkania", value=True)
        btn_run = gr.Button("Transkrybuj i analizuj")

    with gr.Row():
        transcription_output = gr.Textbox(label="Transkrypcja", lines=5, max_lines=20)
        summary_output = gr.HTML(label="📋 Podsumowanie (Markdown)")

    btn_run.click(
        fn=transcribe_and_analyze,
        inputs=[audio_input, create_notes],
        outputs=[transcription_output, summary_output]
    )

ui.title = "🎙️ Transkrypcja i Notatki ze Spotkania"
ui.description = "Transkrypcja za pomocą Whisper + podsumowanie przez GPT-4o."

ui.launch()
        

# interface = gr.Interface(
#     fn=transcribe_and_analyze,
#     inputs=[
#         gr.Audio(type="filepath", label="Wgraj lub nagraj audio", format="wav"),
#         gr.Checkbox(label="Utwórz notatki ze spotkania", value=True),
#     ],
#     outputs=[
#         gr.Textbox(label="Transkrypcja", lines=10, max_lines=20),
#         gr.Markdown(label="📋 Podsumowanie (Markdown)")

#     ],
#     title="🎙️ Transkrypcja i Notatki ze Spotkania",
#     description="Transkrypcja za pomocą Whisper + podsumowanie przez GPT-4o.",
#     examples=[],
#     flagging_mode="never"
# )

# interface.launch(share=True, debug=True, show_error=True)
